In [2]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings("ignore")

nltk.download('gutenberg')
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\capcl\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[!] Skipping model package dependencies and setting `--no-deps`. You don't seem
to have the spaCy package itself installed (maybe because you've built from
source?), so installing the model dependencies would cause spaCy to be
downloaded, which probably isn't what you want. If the model package has other
dependencies, you'll have to install them manually.
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### 1. Converting words or sentences into numeric vectors is fundamental when working with text data. To make sure that you have a solid handle on how these vectors work, generate the TF-IDF vectors for the last three sentences of the example from the beginning of this checkpoint (from the BoW revisited: TF-IDF section).

In [37]:
text = "The best Monty Python sketch is the one about the dead parrot; I laughed so hard. I laugh when I think about Python's Ministry of Silly Walks sketch; it is funny, funny, funny, the best! Chocolate is the best ice cream dessert topping, with a great taste. The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing. I would rather put strawberries on my ice cream for dessert; they have the best taste. The taste of caramel is a fantastic accompaniment to tasty mint ice cream."

In [38]:
# utility function for standard text cleaning
def text_cleaner(text):
    # visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b", " ", text)
    text = ' '.join(text.split())
    return text

In [39]:
text_cleaner(text)

"The best Monty Python sketch is the one about the dead parrot; I laughed so hard. I laugh when I think about Python's Ministry of Silly Walks sketch; it is funny, funny, funny, the best! Chocolate is the best ice cream dessert topping, with a great taste. The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing. I would rather put strawberries on my ice cream for dessert; they have the best taste. The taste of caramel is a fantastic accompaniment to tasty mint ice cream."

In [160]:
nlp = spacy.load('en_core_web_sm')
sample_doc = nlp(text)

In [161]:
sample_sents = [[sent] for sent in sample_doc.sents]
sentences = pd.DataFrame(sample_sents, columns=['text'])
sentences

,text
0,"(The, best, Monty, Python, sketch, is, the, on..."
1,"(I, laugh, when, I, think, about, Python, 's, ..."
2,"(Chocolate, is, the, best, ice, cream, dessert..."
3,"(The, Lumberjack, Song, is, the, funniest, Mon..."
4,"(I, would, rather, put, strawberries, on, my, ..."
5,"(The, taste, of, caramel, is, a, fantastic, ac..."


In [162]:
print(sentences['text'][3])
print(sentences['text'][4])
print(sentences['text'][5])

The Lumberjack Song is the funniest Monty Python bit; I can't think of it without laughing.
I would rather put strawberries on my ice cream for dessert; they have the best taste.
The taste of caramel is a fantastic accompaniment to tasty mint ice cream.


In [177]:
# multiplying TF(IDF) for each word of the last three sentences
Monty = [1*1.585, 0*1.585, 0*1.585 ]
Python = [1*1, 0*1, 0*1 ]
sketch =[1.585*0, 1.585*0, 1.585*0 ]
laugh =[1*1, 1*0, 1*0]
funny = [1.585*1, 1.585*0, 1.585*0]
best = [0*0.585, 1*0.585, 0*0.585]
ice_cream = [ 1.585*0, 1*1.585, 1* 1.585]
dessert = [1.585*0, 1.585*1, 1.585*0]
taste = [0*1, 1*1, 2*1]

In [178]:
print(Monty)
print(Python)
print(sketch)
print(laugh)
print(funny)
print(best)
print(ice_cream)
print(dessert)
print(taste)

[1.585, 0.0, 0.0]
[1, 0, 0]
[0.0, 0.0, 0.0]
[1, 0, 0]
[1.585, 0.0, 0.0]
[0.0, 0.585, 0.0]
[0.0, 1.585, 1.585]
[0.0, 1.585, 0.0]
[0, 1, 2]


### 2. In the 2-grams example above, we only used 2-grams as our features. This time, use both 1-grams and 2-grams together as your feature set. Run the same models in the example and compare the results.

In [179]:
# load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# the chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [ ]:
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)


In [183]:
# group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# combine the sentences from the two novels into one data frame
sentences = pd.DataFrame(alice_sents + persuasion_sents, columns = ["text", "author"])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [184]:
# get rid off stop words and punctuation
# and lemmatize the tokens
for i, sentence in enumerate(sentences["text"]):
    sentences.loc[i, "text"] = " ".join(
        [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

In [185]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5, min_df=2, use_idf=True, norm=u'l2', smooth_idf=True, ngram_range=(1,2))


# applying the vectorizer
X = vectorizer.fit_transform(sentences["text"])

tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([tfidf_df, sentences[["text", "author"]]], axis=1)

# keep in mind that the log base 2 of 1 is 0,
# so a tf-idf score of 0 indicates that the word was present once in that sentence.
sentences.head()

,abide,ability,able,able bear,able persuade,abominate,abroad,absence,absence home,absent,...,young people,young person,young sister,young woman,youth,youth say,zeal,zealous,text,author
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Alice begin tired sit sister bank have twice p...,Carroll
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,remarkable Alice think way hear Rabbit,Carroll
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,oh dear,Carroll


In [186]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

Y = sentences['author']
X = np.array(sentences.drop(['text','author'], 1))

# We split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=123)

# Models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print("----------------------Logistic Regression Scores----------------------")
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

print("----------------------Random Forest Scores----------------------")
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

print("----------------------Gradient Boosting Scores----------------------")
print('Training set score:', gbc.score(X_train, y_train))
print('\nTest set score:', gbc.score(X_test, y_test))

----------------------Logistic Regression Scores----------------------
Training set score: 0.912696063924238

Test set score: 0.8717265867731913
----------------------Random Forest Scores----------------------
Training set score: 0.9786919206865937

Test set score: 0.8695073235685752
----------------------Gradient Boosting Scores----------------------
Training set score: 0.8490677715300384

Test set score: 0.8326675543719485
